In [2]:
from sklearn.neural_network import MLPClassifier
import pickle
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn import preprocessing
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense

In [3]:
root_embedding = './embedding/'

In [4]:
f = open(root_embedding + 'train.pickle', 'rb')
data = pickle.load(f)
f.close()
X_train = data['embeddings']
y_train = data['labels']

In [5]:
f = open(root_embedding + 'test.pickle', 'rb')
data = pickle.load(f)
f.close()
X_test = data['embeddings']
y_test = data['labels']

In [6]:
f = open(root_embedding + 'val.pickle', 'rb')
data = pickle.load(f)
f.close()
X_val = data['embeddings']
y_val = data['labels']

In [7]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)

In [8]:
X_val_minmax = min_max_scaler.fit_transform(X_val)

In [9]:
X_test_minmax = min_max_scaler.transform(X_test)

In [10]:
# print(X_train[:10])
y_train_new = []
for y in y_train:
    match y:
        case 'explorers':
            y_train_new.append(0)
        case 'healers':
            y_train_new.append(1)
        case 'knights':
            y_train_new.append(2)
        case 'wizards':
            y_train_new.append(3)

In [11]:
from sklearn.utils import shuffle
X_train_shuffle, y_train_shuffle = shuffle(X_train, y_train_new, random_state=42)

In [27]:
# classifier = Sequential()

# classifier.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu', input_dim = 512))
# classifier.add(Dense(units = 1024, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dense(units = 1024, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'sigmoid'))
# classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
model = keras.Sequential(
    [
        keras.Input(shape=(512,)),
        layers.Dense(1024),
        layers.Dense(512),
        layers.Dropout(0.5),
        layers.Dense(4),
    ]
)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [28]:
# classifier.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 512)               262656    
                                                                 
 dense_44 (Dense)            (None, 1024)              525312    
                                                                 
 dense_45 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_46 (Dense)            (None, 512)               524800    
                                                                 
 dense_47 (Dense)            (None, 128)               65664     
                                                                 
 dense_48 (Dense)            (None, 4)                 516       
                                                                 
Total params: 2,428,548
Trainable params: 2,428,548
No

In [ ]:
# classifier.fit(X_train, y_train_new, batch_size = 10, epochs = 100, verbose = 0)

In [15]:
model.fit(X_train_shuffle, y_train_shuffle, batch_size = 10, epochs = 100, verbose = 0)

In [16]:
y_test_new = []
for y in y_test:
    match y:
        case 'explorers':
            y_test_new.append(0)
        case 'healers':
            y_test_new.append(1)
        case 'knights':
            y_test_new.append(2)
        case 'wizards':
            y_test_new.append(3)

In [17]:
prediction = model.predict(X_test)

65/65 [==============================] - 6s 5ms/step


In [18]:
model.evaluate(X_test, y_test_new)

65/65 [==============================] - 1s 4ms/step - loss: 1.5292 - sparse_categorical_accuracy: 0.2705


[1.5291980504989624, 0.2704678475856781]

In [19]:
model.save('./ANNModel_new.h5', overwrite=True, save_format='h5')